In [34]:
import numpy as np
import pandas as pd
import os

In [45]:
folder = sorted(os.listdir('my_naics5_chg'))
uniquecodes = pd.Series(folder[2:]).str[6:8].unique()
uniquecodes

array(['31', '32', '33', '51', '52', '54', '61', '62', '71'], dtype=object)

In [41]:
umask = temp_df['type_kurt20'] == 'Metro'
rmask = temp_df['type_kurt20'] == 'Rural'
temp_df[umask][temp_df.filter(regex='annual').columns].sum()

annual_avg_emplvl_90    245620.0
annual_avg_emplvl_00    218265.0
annual_avg_emplvl_10    139324.0
annual_avg_emplvl_20     62879.0
dtype: float64

In [42]:
data = []
code = []

for file in folder:
    if '51' in file[6:8]:
        temp_df = pd.read_csv('my_naics5_chg/' + file)
        umask = temp_df['type_kurt20'] == 'Metro'
        rmask = temp_df['type_kurt20'] == 'Rural'
        data.append(pd.concat([pd.Series(file[12:-4], index=['name']),
                               temp_df[temp_df.filter(regex='annual').columns].sum(),
                               temp_df[umask][temp_df.filter(regex='annual').columns].sum(),
                               temp_df[rmask][temp_df.filter(regex='annual').columns].sum()]))
        code.append(file[6:11])
    else:
        pass

df = pd.DataFrame(dict(zip(code, data))).T    

In [49]:
for n in uniquecodes:
    data = []
    code = []
    for file in folder:
        if n in file[6:8]:
            temp_df = pd.read_csv('my_naics5_chg/' + file)
            umask = temp_df['type_kurt20'] == 'Metro'
            rmask = temp_df['type_kurt20'] == 'Rural'
            data.append(pd.concat([pd.Series(file[12:-4], index=['name']),
                                   temp_df[temp_df.filter(regex='annual').columns].sum(),
                                   temp_df[umask][temp_df.filter(regex='annual').columns].sum(),
                                   temp_df[rmask][temp_df.filter(regex='annual').columns].sum()]))
            code.append(file[6:11])
        else:
            pass

    pd.DataFrame(dict(zip(code, data))).T.to_csv('analysis/industry_naics_' + n + '.csv')

In [44]:
df.to_csv('analysis/industry_naics_' + '51' + '.csv')

In [26]:
# rate of change function
def rate_chg(df, year1, year2, chg):
    df[chg] = np.where((df[year1]== 0),
                       (df[year2] - df[year1]) / 1,
                       (df[year2] - df[year1]) / df[year1])
    
# define new column namer
empl_cols = df.filter(regex='annual').columns
namer = empl_cols.str.split('_')

# calculate rate of change
for x in range(0,3):
    rate_chg(df,
             empl_cols[x],
             empl_cols[x+1],
             'chg_' + namer[x][2] + '_' + namer[x][3] + '_' + namer[x+1][3])
    
# calculate total rate of change column (1990-2020)
df['chg_emplvl_90_20'] = np.where((df['annual_avg_emplvl_90']==0),
                                 ((df['annual_avg_emplvl_20'] - df['annual_avg_emplvl_90']) / 1),
                                 ((df['annual_avg_emplvl_20'] - df['annual_avg_emplvl_90']) / df['annual_avg_emplvl_90']))

In [ ]:
# read dataframes, calculates and stores totals by geography and region
path = 'my_naics5_chg/naics_'
codes = ['11', '21', '31', '51', '52', '54']

# empty lists for processing
shell = []
names = []

for code in codes:
    df = pd.read_csv(path + code + '.csv')

    classification = 'type_census20' # classification scheme
    
    temp_list = []
    for col in df.columns[8:12]:
        s = df.groupby(classification)[col].sum()
        temp_list.append(s)
    mynaics = pd.concat(temp_list, axis=1)
    
    # store values in dictionary
    shell.append(mynaics)
    names.append('geo_naics' + code)
d1 = dict(zip(names, shell))

In [ ]:
# calculate regional totals and save to dictionary

# clear lists
shell = []
names = []

for code in codes:
    df = pd.read_csv(path + code + '.csv')
    
    temp_list = []
    for col in df.columns[8:12]:
        s = df.groupby('region')[col].sum()
        temp_list.append(s)
    mynaics = pd.concat(temp_list, axis=1)
    
    # store values in dictionary
    shell.append(mynaics)
    names.append('region_naics' + code)
d2 = dict(zip(names, shell))

In [ ]:
for code in codes:
    print(d1['geo_naics' + code].sum() - d2['region_naics' + code].sum())

In [ ]:
# calculate state totals (preserving region) and save to dictionary

# clear lists
shell = []
names = []

for code in codes:
    temp_list = []
    for col in df.columns[8:12]:
        s = df.groupby(['State', 'region'], as_index=False)[col].sum()
        temp_list.append(s)
    mynaics = pd.concat(temp_list, axis=1)
    
    # drop duplicate state and region cols
    mynaics = mynaics.iloc[:,~mynaics.columns.duplicated()]
    
    # store values in dictionary
    shell.append(mynaics)
    names.append('state_naics' + code)
d3 = dict(zip(names, shell))

In [ ]:
# create total jobs row
dicts = [d1, d2]
for d in dicts:
    for key in d.keys():
        d[key] = pd.concat([d[key],
                            pd.DataFrame([d[key].sum()], index=['Total'], columns=d[key].columns),])

In [ ]:
# create rate of change function
def rate_chg(df, year1, year2, chg):
    df[chg] = np.where((df[year1]==0),
                       ((df[year2] - df[year1]) / 1).round(4),
                       ((df[year2] - df[year1]) / df[year1]).round(4))
    
# create geographical and regional national job share and change columns
dicts = [d1, d2]
for d in dicts:
    for key in d.keys():
        # job share (removing total jobs row)
        for col in d[key].columns:
            d[key]['emplpct_' + col[-2:]] = (d[key].iloc[:-1][col] /
                                             d[key].iloc[:-1][col].sum()).round(4)

        # define % change column namer
        namer = d[key].columns.str.split('_')
        for x in range(0,3):
            
            # % change 90-00, 00-10, 10-20
            rate_chg(d[key], d[key].columns[x], d[key].columns[x+1], 'pct_chg_' + namer[x][3] + '_' + namer[x+1][3])
            
        # overall % change 90-20
        d[key]['pct_chg_90_20'] = np.where((d[key]['annual_avg_emplvl_90'] == 0),
                                          ((d[key]['annual_avg_emplvl_20'] - d[key]['annual_avg_emplvl_90']) / 1).round(4),
                                          ((d[key]['annual_avg_emplvl_20'] - d[key]['annual_avg_emplvl_90']) / d[key]['annual_avg_emplvl_90']).round(4))

In [ ]:
## issues with the pct change function when calculating regiona and geo totals

In [ ]:
# do same for states, make some tweaks for the different df structure
for key in d3.keys():
    # job share
    for col in d3[key].columns[2:]:
        d3[key]['emplpct_' + col[-2:]] = (d3[key].iloc[:-1][col] / 
                                          d3[key].iloc[:-1][col].sum()).round(4)

    # define % change column namer
    namer = d3[key].columns.str.split('_')
    for x in range(0,3):
            
        # % change 90-00, 00-10, 10-20
        rate_chg(d3[key], d3[key].columns[2:][x], d3[key].columns[2:][x+1], 'pct_chg_' + namer[x+2][3] + '_' + namer[x+3][3])
            
        # overall % change 90-20
        d3[key]['pct_chg_90_20'] = np.where((d3[key]['annual_avg_emplvl_90'] == 0),
                                           ((d3[key]['annual_avg_emplvl_20'] - d3[key]['annual_avg_emplvl_90']) / 1).round(4),
                                           ((d3[key]['annual_avg_emplvl_20'] - d3[key]['annual_avg_emplvl_90']) / d3[key]['annual_avg_emplvl_90']).round(4))

In [ ]:
# export to CSVs
for key in d1.keys():
    d1[key].to_csv('analysis/geo_naics_' + key[-2:] + '.csv',) #index_label=False)

for key in d2.keys():
    d2[key].drop('Other').to_csv('analysis/region_naics_' + key[-2:] + '.csv',)# index_label=False)
    
for key in d3.keys():
    d3[key].to_csv('analysis/state_naics_' + key[-2:] + '.csv',)# index_label=False)